In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [10]:
players = pd.read_csv("Data/nba_players_stats/Players.csv")

In [12]:
players[-10:]

,Unnamed: 0,Player,height,weight,collage,born,birth_city,birth_state
3912,3912,Denzel Valentine,198.0,96.0,Michigan State University,1993.0,Lansing,Michigan
3913,3913,Fred VanVleet,183.0,88.0,Wichita State University,1994.0,Rockford,Illinois
3914,3914,Taurean Waller-Prince,183.0,88.0,NaN,1994.0,NaN,NaN
3915,3915,Okaro White,203.0,92.0,Florida State University,1992.0,Clearwater,Florida
3916,3916,Isaiah Whitehead,193.0,96.0,Seton Hall University,1995.0,Brooklyn,New York
3917,3917,Troy Williams,198.0,97.0,South Carolina State University,1969.0,Columbia,South Carolina
3918,3918,Kyle Wiltjer,208.0,108.0,Gonzaga University,1992.0,Portland,Oregon
3919,3919,Stephen Zimmerman,213.0,108.0,"University of Nevada, Las Vegas",1996.0,Hendersonville,Tennessee
3920,3920,Paul Zipser,203.0,97.0,NaN,1994.0,Heidelberg,Germany
3921,3921,Ivica Zubac,216.0,120.0,NaN,1997.0,Mostar,Bosnia and Herzegovina


In [5]:
season_stats = pd.read_csv("Data/nba_players_stats/Seasons_Stats.csv")

In [9]:
season_stats[-1:]

,Unnamed: 0,Year,Player,Pos,Age,Tm,G,GS,MP,PER,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
24690,24690,2017.0,Ivica Zubac,C,19.0,LAL,38.0,11.0,609.0,17.0,...,0.653,41.0,118.0,159.0,30.0,14.0,33.0,30.0,66.0,284.0


In [13]:
season_stats.columns

Index([u'Unnamed: 0', u'Year', u'Player', u'Pos', u'Age', u'Tm', u'G', u'GS',
       u'MP', u'PER', u'TS%', u'3PAr', u'FTr', u'ORB%', u'DRB%', u'TRB%',
       u'AST%', u'STL%', u'BLK%', u'TOV%', u'USG%', u'blanl', u'OWS', u'DWS',
       u'WS', u'WS/48', u'blank2', u'OBPM', u'DBPM', u'BPM', u'VORP', u'FG',
       u'FGA', u'FG%', u'3P', u'3PA', u'3P%', u'2P', u'2PA', u'2P%', u'eFG%',
       u'FT', u'FTA', u'FT%', u'ORB', u'DRB', u'TRB', u'AST', u'STL', u'BLK',
       u'TOV', u'PF', u'PTS'],
      dtype='object')